# Exploring eBay Car Sales Data
In this project, we will work with a dataset of used cars from *eBay Kleinanzeigen*, a classifieds section of the German eBay website. This dataset was originally uploaded by [orgesleka](https://www.kaggle.com/orgesleka) (not available anymore).

we have made a modification from the original dataset, by sampling 50,000 data points from the full dataset, to ensure our code runs quickly in our hosted environment.

**GOAL**: Get a better grasp on data cleaning process.
## 1. Opening and Observing the Dataset
Now let's open our dataset (the dataset doesn't work with default `UTF-8` encoding, so we used `Latin-1` encoding instead).

In [6]:
import numpy as np
import pandas as pd

autos = pd.read_csv("autos.csv", encoding = "Latin-1")
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

We have retrieved the column information of the dataset. The dataset contains 20 column, and some of them that will be useful for our analysis with detailed description as follows:
- `dataCrawled` - When this ad was first crawled. All field-values are taken from this date.
- `name` - Name of the car.
- `seller` - Whether the seller is private or a dealer.
- `offerType` - The type of listing.
- `price` - The price on the ad to sell the car.
- `abtest` - Whether the listing is included in a A/B test.
- `vehicleType` - The vehicle Type.
- `yearOfRegistration` - The year in which the car was first registered.
- `gearbox` - The transmission type.
- `powerPS` - The power of the car in PS.
- `model` - The car model name.
- `kilometer` - How many kilometers the car has driven.
- `monthOfRegistration` - The month in which the car was first registed.

After observing the `info` for a while, we noticed that some columns have null values (we will decide what to do to those data points later). And that the column names use *camelcase* instead of Python's preferred *snakecase*, which means we cannot just replace spaces with underscores.

Now let's preview the first 5 data points on our dataset, to get a better understanding.

In [7]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## 2. Cleaning the Data
### 2.1 Converting *camelcase* into *snakecase*
First, we convert the column name from *camelcase* to *snakecase* and reword some of the column names to be more descriptive. We will use `dataFrame.rename()` method for this, set the `axis = 1` to indentify that we want to rename our first row, and set `inplace = True` to automatically assign the changes to our dataset.

In [9]:
autos.rename({"dateCrawled":"ad_crawled",
             "offerType":"offer_type",
             "vehicleType":"vehicle_type",
             "yearOfRegistration":"registration_year",
             "powerPS":"power_ps",
             "monthOfRegistration":"registration_month",
             "fuelType":"fuel_type",
             "notRepairedDamage":"unrepaired_damage",
             "dateCreated":"ad_created",
             "nrOfPictures":"nbr_pictures",
             "postalCode":"postal_code",
             "lastSeen":"last_seen"},
             axis=1, inplace=True)

In [11]:
autos.head(1)

,ad_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nbr_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54


Nice! now our column names are using *snakecase* format. Let's explore our dataset again to find out if other cleaning process is required. Things that will interest us:
- Any columns that have mostly one value that are candidates to be dropped (not useful for our analysis).
- Any columns that need more investigation.
- Any columns that should store numeric values, but string instead.

In [13]:
autos.describe(include="all")

,ad_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nbr_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-02 11:37:04,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


### 2.2 Removing irrelevant data
After further investigation, we found out that column `seller`, `offer_type`, `abtest`, and `nbr_pictures` very skewed data, or no data at all. So we will disregard those columns later.

In [27]:
autos["seller"].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

In [28]:
autos["nbr_pictures"].value_counts()

0    50000
Name: nbr_pictures, dtype: int64

In [62]:
autos = autos.drop(["seller", "offer_type", "abtest", "nbr_pictures"], axis=1)

### 2.3 Converting string to numerical values
Next, we also noticed that column `price` and `odometer` should store numerical values, but based on preview above, both columns stored string instead. We have to convert them.

In [29]:
autos[["price", "odometer"]].head(2)

,price,odometer
0,"$5,000","150,000km"
1,"$8,500","150,000km"


In the column `price` we will remove the dollar ($) symbol, and the comma (,) symbol. Then we store it as `int`. We also need to rename the column name as `price_usd`.

In [42]:
autos["price"] = (autos["price"].str.replace("$", "")
                 .str.replace(",", "")
                 .astype(int))
autos.rename({"price":"price_usd"}, axis=1, inplace=True)

For `odometer`, we need to remove the "km" and comma (,) symbol, change the type to `int`, and rename the column name as `odometer_km`.

In [37]:
autos["odometer"] = (autos["odometer"].str.replace("km", "")
                 .str.replace(",", "")
                 .astype(int))
autos.rename({"odometer":"odometer_km"}, axis=1, inplace=True)

Let's check our final results.

In [43]:
autos[["price_usd", "odometer_km"]].head(2)

,price_usd,odometer_km
0,5000,150000
1,8500,150000


### 2.3 Removing outliers
Now that we have integer values on `price_usd` and `odometer_km` columns, we can further check them for any unrealistically high or low values (outliers) that we might want to remove.

For each column we will use:
- `Series.unique().shape` to see how many unique values.
- `Series.describe()` to view min/max/median/mean/etc.
- `Series.value_counts()` to see the frequency of each unique values

In [70]:
print(autos["price_usd"].unique().shape)
print(autos["price_usd"].describe())
print(autos["price_usd"].value_counts().head(20))

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_usd, dtype: float64
0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price_usd, dtype: int64


From result above, we can see that the price is most likely been rounded. But since we know there are 2357 different values, the rounding most likely being done by the seller, not a pre-set options by eBay platform.

We can also see that there are a lot of $0 (1421 occurrences). Given eBay is an auction platform, it is legitimate for opening bid price to be as low as possible. We can choose to leave them as is, but since they may skew our statistic, we choose to drop them instead. We consider this decision safe because 1421 occurrences is less than 2% of our whole dataset.

Let's check for the high prices.

In [72]:
print(autos["price_usd"].value_counts().sort_index(ascending=False).head(10))

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
Name: price_usd, dtype: int64


Those values seem urealistically high. We will limit our analysis